# Baseline Model - Infostop with GPS positions

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import time
from infostop import Infostop
import pyproj
import sklearn
import pickle
import torch
sys.path.append('./src')
from data_utils import *
clear_output(wait=False)

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [3]:
classification_test_performance = []
training_time = []
prediction_time = []
number_list=np.arange(0,12).tolist()
random.Random(17).shuffle(number_list)
print(number_list)
#number_list = [10, 4, 6, 8, 1, 9, 3, 7, 5, 0, 2, 11] #already randomized

[10, 3, 0, 2, 7, 11, 1, 9, 5, 4, 6, 8]


### 4-fold Infostop

In [4]:
seq_cutoff_speed = 45
seq_cutoff_time = 60
filter_seq = 5
for k in range(0,12):
    print(f'Train partition: {number_list[0:k]+number_list[k+1:]}')
    print(f'Test partition: {number_list[k]}')
    user_train = train = number_list[0:k]+number_list[k+1:]
    user_test = test = [number_list[k]]
    
    #data_train = pd.concat([create_data_frame(*load_user_data(user,load_web_mercator = True, load_GPS = True), segmentation=True, seq_cutoff_time = seq_cutoff_time, seq_cutoff_speed = seq_cutoff_speed) for user in user_train]).reset_index(drop=True)
    data_test = pd.concat([create_data_frame(*load_user_data(user,load_web_mercator = True, load_GPS = True), segmentation=True, seq_cutoff_time = seq_cutoff_time, seq_cutoff_speed = seq_cutoff_speed) for user in user_test]).reset_index(drop=True)
    #data_train = data_train[data_train['segment_ix'] >= filter_seq]
    data_test = data_test[data_test['segment_ix'] >= filter_seq]
    model = Infostop()
    startPrediction = time.time()
    labels = model.fit_predict(data_test[['lon', 'lat']].values)
    endPrediction = time.time()
    prediction_time.append(endPrediction-startPrediction)
    pred = np.zeros_like(labels)
    pred[labels >= 0] = 1
    cr=classification_report(data_test['label'], pred, target_names = ['Motion','Stop'],output_dict=True)
    classification_test_performance.append(cr)
F1_macro_AVG = []

Train partition: [3, 0, 2, 7, 11, 1, 9, 5, 4, 6, 8]
Test partition: 10
Train partition: [10, 0, 2, 7, 11, 1, 9, 5, 4, 6, 8]
Test partition: 3
Train partition: [10, 3, 2, 7, 11, 1, 9, 5, 4, 6, 8]
Test partition: 0
Train partition: [10, 3, 0, 7, 11, 1, 9, 5, 4, 6, 8]
Test partition: 2
Train partition: [10, 3, 0, 2, 11, 1, 9, 5, 4, 6, 8]
Test partition: 7
Train partition: [10, 3, 0, 2, 7, 1, 9, 5, 4, 6, 8]
Test partition: 11
Train partition: [10, 3, 0, 2, 7, 11, 9, 5, 4, 6, 8]
Test partition: 1
Train partition: [10, 3, 0, 2, 7, 11, 1, 5, 4, 6, 8]
Test partition: 9
Train partition: [10, 3, 0, 2, 7, 11, 1, 9, 4, 6, 8]
Test partition: 5
Train partition: [10, 3, 0, 2, 7, 11, 1, 9, 5, 6, 8]
Test partition: 4
Train partition: [10, 3, 0, 2, 7, 11, 1, 9, 5, 4, 8]
Test partition: 6
Train partition: [10, 3, 0, 2, 7, 11, 1, 9, 5, 4, 6]
Test partition: 8


In [5]:
np.save('classification_test_performanceClust12fold.npy',classification_test_performance)
F1_macro_AVG = []
F1_weighted_AVG = []
for cr in classification_test_performance:
    F1_macro_AVG.append(cr['macro avg']['f1-score'])
    F1_weighted_AVG.append(cr['weighted avg']['f1-score'])
supports = [cf['macro avg']['support'] for cf in classification_test_performance]
print(f'MEAN = {np.mean(F1_macro_AVG)}, WEIGHTED AVG = {np.sum([F1_macro_AVG[i]*supports[i] for i in range(0,len(F1_macro_AVG))])/(np.sum(supports))}, STDEV = {np.std(F1_macro_AVG)}')
print(f'MEAN = {np.mean(F1_weighted_AVG)}, WEIGHTED AVG = {np.sum([F1_weighted_AVG[i]*supports[i] for i in range(0,len(F1_weighted_AVG))])/(np.sum(supports))}, STDEV = {np.std(F1_weighted_AVG)}')

MEAN = 0.602154591003217, WEIGHTED AVG = 0.6535989161830201, STDEV = 0.14739750561616363
MEAN = 0.6829053597361155, WEIGHTED AVG = 0.7639122059331971, STDEV = 0.15659593943266806


In [6]:
results = [i for i in zip(F1_macro_AVG, F1_weighted_AVG, supports)]

In [7]:
np.save('resClust12fold.npy',results)

In [8]:
np.sum(prediction_time)

11.659857511520386